In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import os
import urllib
from urllib.request import urlopen
import pickle
import sys
import numpy as np
import pandas as pd
import elementpath
import re


In [2]:
# Define the user agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
]

## scrape Putin's messages 2009-2023

In [3]:
# Define the base URL and path for the desired page
# I can loop only until messages! 
base_url = 'http://kremlin.ru'
path = '/events/president/transcripts/messages/'

In [4]:
urls = []

# Loop over 8 pages
for i in range(len(user_agents)):
    headers = {'User-Agent': user_agents[i]}
    url = base_url + path
    response = requests.get(url, headers=headers)

    # Check the status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # TODO: Scrape the URLs from the page and save them to a list
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.startswith('/'):
                urls.append(base_url + href)
                print(base_url + href)  # Print the URL while it works
    else:
        print(
            f'Request for {url} failed with status code {response.status_code}')

    # Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
    time.sleep(1 + 4 * random.random())

http://kremlin.ru/
http://kremlin.ru/events/president/news
http://kremlin.ru/structure/president
http://kremlin.ru/multimedia/video
http://kremlin.ru/acts/news
http://kremlin.ru/contacts
http://kremlin.ru/search
http://kremlin.ru/acts/bank
http://kremlin.ru/events/president/news
http://kremlin.ru/events/administration
http://kremlin.ru/events/state-council
http://kremlin.ru/events/security-council
http://kremlin.ru/events/councils
http://kremlin.ru/events/president/news
http://kremlin.ru/events/president/transcripts
http://kremlin.ru/events/president/trips
http://kremlin.ru/events/president/letters
http://kremlin.ru/structure/president
http://kremlin.ru/events/president/transcripts
http://kremlin.ru/events/president/transcripts/messages
http://kremlin.ru/events/president/transcripts/statements
http://kremlin.ru/events/president/transcripts/deliberations
http://kremlin.ru/events/president/transcripts/speeches
http://kremlin.ru/events/president/transcripts/comminity_meetings
http://kreml

In [5]:
for link in soup.find_all('a'):
    print(link.get('href'))

#sidebar
/
/events/president/news
/structure/president
/multimedia/video
/acts/news
/contacts
/search
None
/acts/bank
#
#clear
#
#clear
http://pravo.gov.ru/
https://www.may9.ru/
/events/president/news
/events/administration
/events/state-council
/events/security-council
/events/councils
#focus
/events/president/news
/events/president/transcripts
/events/president/trips
/events/president/letters
/structure/president
/events/president/transcripts
/events/president/transcripts/messages
/events/president/transcripts/statements
/events/president/transcripts/deliberations
/events/president/transcripts/speeches
/events/president/transcripts/comminity_meetings
/events/president/transcripts/press_conferences
/events/president/transcripts/interviews
/events/president/transcripts/articles
/events/president/transcripts/messages/70565
/events/president/transcripts/messages/70565
/events/president/transcripts/messages/70565/photos
/events/president/transcripts/messages/70565/videos
/events/president

In [ ]:
#increase the recursion limit
sys.setrecursionlimit(10**6)

In [6]:
soup.find_all('a')[28].get('href')

'/events/president/transcripts/messages'

In [6]:
# Find all links with an href attribute that starts with "/events/president/transcripts/messages"
messages_links = soup.find_all("a", href=lambda href: href and href.startswith("/events/president/transcripts/messages"))

In [7]:
base_url = 'http://kremlin.ru'

# Find all links with an href attribute that starts with "/events/president/transcripts/messages"
messages_links = soup.find_all("a", href=lambda href: href and href.startswith("/events/president/transcripts/messages"))

# convert the messages_links to a set to remove duplicates
messages_links = set(messages_links)

# create an empty list to store the urls
urls = []

# loop through the messages_links to create the urls
for link in messages_links:
    parts = link["href"].split("/")
    last_part = parts[-1]
    if last_part.isdigit() and len(last_part) == 5 or len(last_part) == 4:
        url = base_url + link["href"]
        urls.append(url)

urls = set(urls)
for url in urls:
    print(url)

http://kremlin.ru/events/president/transcripts/messages/62582
http://kremlin.ru/events/president/transcripts/messages/50864
http://kremlin.ru/events/president/transcripts/messages/19825
http://kremlin.ru/events/president/transcripts/messages/17118
http://kremlin.ru/events/president/transcripts/messages/53379
http://kremlin.ru/events/president/transcripts/messages/56957
http://kremlin.ru/events/president/transcripts/messages/5979
http://kremlin.ru/events/president/transcripts/messages/65418
http://kremlin.ru/events/president/transcripts/messages/47173
http://kremlin.ru/events/president/transcripts/messages/59863
http://kremlin.ru/events/president/transcripts/messages/1968
http://kremlin.ru/events/president/transcripts/messages/70565
http://kremlin.ru/events/president/transcripts/messages/14088
http://kremlin.ru/events/president/transcripts/messages/9637


In [9]:
urls = list(urls)
len(urls)

14

In [11]:
# make a soup for each url of 14urls object

soup_list = []
processed_urls = []

for url in urls:
    # Preprocess the URL
    processed_url = url.strip()

    # Check if the URL has already been processed
    if processed_url in processed_urls:
        print(f"Skipping {processed_url} - already processed")
        continue

    try:
        # Make a GET request to the desired URL with rotating user agents and sleep time
        i = 0
        while True:
            headers = {'User-Agent': user_agents[i]}
            response = requests.get(processed_url, headers=headers, timeout=10)

            if response.status_code == 200:
                # make a soup object from the response content
                soup = BeautifulSoup(response.content, 'html.parser')
                soup_list.append(soup)

                # Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
                time.sleep(1 + 4 * random.random())
                i = (i + 1) % len(user_agents)
                processed_urls.append(processed_url)
                print(f"Processed URL: {processed_url}")
                break

            else:
                # If the response is not 200, sleep for a random time between 1 and 5 seconds and try again with a different user agent
                time.sleep(1 + 4 * random.random())
                i = (i + 1) % len(user_agents)

    except Exception as e:
        print(f"Error getting data from {processed_url}: {e}")

Processed URL: http://kremlin.ru/events/president/transcripts/messages/62582
Processed URL: http://kremlin.ru/events/president/transcripts/messages/50864
Processed URL: http://kremlin.ru/events/president/transcripts/messages/19825
Processed URL: http://kremlin.ru/events/president/transcripts/messages/17118
Processed URL: http://kremlin.ru/events/president/transcripts/messages/53379
Processed URL: http://kremlin.ru/events/president/transcripts/messages/56957
Processed URL: http://kremlin.ru/events/president/transcripts/messages/5979
Processed URL: http://kremlin.ru/events/president/transcripts/messages/65418
Processed URL: http://kremlin.ru/events/president/transcripts/messages/47173
Processed URL: http://kremlin.ru/events/president/transcripts/messages/59863
Processed URL: http://kremlin.ru/events/president/transcripts/messages/1968
Processed URL: http://kremlin.ru/events/president/transcripts/messages/70565
Processed URL: http://kremlin.ru/events/president/transcripts/messages/14088
P

In [12]:
len(processed_urls)

14

In [14]:
#increase the recursion limit
sys.setrecursionlimit(10**6)

In [17]:
# Save the soup object to a file using pickle
with open("data/corpus_adriana/corpus_president_of_russia/messages_soup_list.pkl", "wb") as f:
    pickle.dump(soup_list, f)

In [38]:
# Read the soup-file
with open("data/corpus_adriana/corpus_president_of_russia/messages_soup_list.pkl", "rb") as f:
    soup_list = pickle.load(f)

### find single elements in each soup

In [19]:
# datetime
for soup in soup_list:
    print(soup.find('time', {'itemprop': 'datePublished'})['datetime'])

2020-01-15
2015-12-03
2013-12-12
2012-12-12
2016-12-01
2018-03-01
2009-11-12
2021-04-21
2014-12-04
2019-02-20
2008-11-05
2023-02-21
2011-12-22
2010-11-30


In [20]:
# sorted date
sorted_soup_list = sorted(soup_list, key=lambda soup: soup.find(
    'time', {'itemprop': 'datePublished'})['datetime'])
for soup in sorted_soup_list:
    print(soup.find('time', {'itemprop': 'datePublished'})['datetime'])

2008-11-05
2009-11-12
2010-11-30
2011-12-22
2012-12-12
2013-12-12
2014-12-04
2015-12-03
2016-12-01
2018-03-01
2019-02-20
2020-01-15
2021-04-21
2023-02-21


In [39]:
# url
for soup in soup_list:
    url = soup.find('div', {'class': 'read'}).get('data-href')
    print(f"{url}")

http://kremlin.ru/events/president/transcripts/messages/62582
http://kremlin.ru/events/president/transcripts/messages/50864
http://kremlin.ru/events/president/transcripts/messages/19825
http://kremlin.ru/events/president/transcripts/messages/17118
http://kremlin.ru/events/president/transcripts/messages/53379
http://kremlin.ru/events/president/transcripts/messages/56957
http://kremlin.ru/events/president/transcripts/messages/5979
http://kremlin.ru/events/president/transcripts/messages/65418
http://kremlin.ru/events/president/transcripts/messages/47173
http://kremlin.ru/events/president/transcripts/messages/59863
http://kremlin.ru/events/president/transcripts/messages/1968
http://kremlin.ru/events/president/transcripts/messages/70565
http://kremlin.ru/events/president/transcripts/messages/14088
http://kremlin.ru/events/president/transcripts/messages/9637


In [41]:
soup= soup_list[10]
url = soup.find('div', {'class': 'read'}).get('data-href')
print(f"{url}")

http://kremlin.ru/events/president/transcripts/messages/1968


In [22]:
## extract the title
for soup in soup_list:
    print(soup.find('title').text.strip())

Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Федеральному Собранию Российской Федерации • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Федеральному Собранию Российской Федерации • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России
Послание Президента Федеральному Собранию • Президент России


In [35]:
for i, soup in enumerate(soup_list):
    url = soup.find('div', {'class': 'read'}).get('data-href')
    speakers = set()
    for p in soup.find_all('p'):
        speaker_tag = p.find('b')
        if speaker_tag is not None:
            speakers.add(speaker_tag.text)
    print(f"{i+1}. Number of speakers in {url}: {len(speakers)}")


1. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/62582: 2
2. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/50864: 1
3. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/19825: 2
4. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/17118: 2
5. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/53379: 1
6. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/56957: 2
7. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/5979: 1
8. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/65418: 1
9. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/47173: 2
10. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/59863: 2
11. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/1968: 0
12. Number

In [44]:
soup = soup_list[10]
url = soup.find('div', {'class': 'read'}).get('data-href')
speakers = set()

for p in soup.find_all('p'):
    if 'Д.Медведев:' in p.text:
        if p.find('b') is None:
            new_tag = soup.new_tag('b')
            new_tag.string = 'Д.Медведев:'
            p.string.replace_with(new_tag)
            speakers.add(new_tag.string)

In [45]:
for i, soup in enumerate(soup_list):
    url = soup.find('div', {'class': 'read'}).get('data-href')
    speakers = set()
    for p in soup.find_all('p'):
        speaker_tag = p.find('b')
        if speaker_tag is not None:
            speakers.add(speaker_tag.text)
    print(f"{i+1}. Number of speakers in {url}: {len(speakers)}")

1. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/62582: 2
2. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/50864: 1
3. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/19825: 2
4. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/17118: 2
5. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/53379: 1
6. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/56957: 2
7. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/5979: 1
8. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/65418: 1
9. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/47173: 2
10. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/59863: 2
11. Number of speakers in http://kremlin.ru/events/president/transcripts/messages/1968: 1
12. Number

In [46]:
soup = soup_list[10]
for p in soup.find_all('p'):
    print(p.text)

Все материалы сайта доступны по лицензии:
Creative Commons Attribution 4.0 International
Д.Медведев:
Уважаемые депутаты и члены Совета Федерации!
Начну своё Послание с оценки событий текущего года. В 2008 году в нашей стране произошло обновление ключевых властных институтов. По итогам выборов Президента было сформировано новое Правительство. В полную силу заработали в новой Государственной Думе парламентские партии.
Началась реализация новых планов долгосрочного развития экономики и социальной сферы. Строятся заводы и дороги. Перевооружаются армия и флот. Осваиваются новые технологии. Создаются учебные, научные, медицинские, центры. Наши спортсмены показывают примеры ярких побед.
Но этот год стал для наших граждан, для всех нас не только временем новых надежд и достижений. Произошли события, ставшие, я уверен, очень значимыми буквально для каждого в нашей стране. И одновременно серьёзным испытанием для всей России.
Это варварская агрессия против Южной Осетии. И, конечно, набирающий обо

In [50]:

# create an empty list to store data
data = []

# iterate through each soup
for soup in soup_list:
    # extract data
    date = soup.find('time', {'itemprop': 'datePublished'})['datetime']
    url = soup.find('div', {'class': 'read'}).get('data-href')
    title = soup.find('h1').text
    speakers = set()
    text = ""

    # iterate through each paragraph and check for speaker
    for p in soup.find_all('p'):
        speaker_tag = p.find('b')
        if speaker_tag is not None:
            speakers.add(speaker_tag.text)
        text += p.text.strip() + " "

    # append data to the list
    data.append([date, url, title, ", ".join(speakers), text])

# create a dataframe
mtfa = pd.DataFrame(data, columns=['date', 'URL', 'title', 'speaker', 'text'])


In [49]:
#check if whole text was extracted
for soup in soup_list:
    date = soup.find('time', {'itemprop': 'datePublished'})['datetime']
    url = soup.find('div', {'class': 'read'}).get('data-href')
    title = soup.find('h1').text.strip()
    speakers = set()
    entire_text = ''
    for p in soup.find_all('p'):
        speaker_tag = p.find('b')
        if speaker_tag is not None:
            speakers.add(speaker_tag.text)
        entire_text += p.text.strip() + '\n'
    print(f"Date: {date}\nURL: {url}\nTitle: {title}\nSpeaker(s): {speakers}\nEntire text: {entire_text}\n")


Date: 2020-01-15
URL: http://kremlin.ru/events/president/transcripts/messages/62582
Title: Послание Президента Федеральному Собранию
Speaker(s): {'* * *', 'В.Путин:'}
Entire text: Все материалы сайта доступны по лицензии:
Creative Commons Attribution 4.0 International
Владимир Путин обратился с Посланием к Федеральному Собранию. Церемония оглашения прошла в Москве, в Центральном выставочном зале «Манеж».
На церемонии присутствовали около 1300 человек: члены Совета Федерации, депутаты
Государственной Думы, члены Правительства, руководители Конституционного и Верховного судов, губернаторский корпус, председатели законодательных собраний
субъектов Федерации, главы традиционных конфессий, общественные деятели, в том
числе главы общественных палат регионов, руководители крупнейших средств
массовой информации.
* * *
В.Путин: Уважаемые
члены Совета Федерации и депутаты Государственной Думы! Уважаемые граждане
России!
Послание Федеральному
Собранию впервые оглашается в самом начале года. Нам н

In [51]:
mtfa

,date,URL,title,speaker,text
0,2020-01-15,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
1,2015-12-03,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...
2,2013-12-12,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
3,2012-12-12,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
4,2016-12-01,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...
5,2018-03-01,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
6,2009-11-12,http://kremlin.ru/events/president/transcripts...,Послание Федеральному Собранию Российской Феде...,Д.Медведев:,Все материалы сайта доступны по лицензии: Crea...
7,2021-04-21,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...
8,2014-12-04,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
9,2019-02-20,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...


In [52]:
#save the df as pickle
mtfa.to_pickle('data/corpus_adriana/corpus_president_of_russia/messages_to_federal_assembly.pkl')

In [53]:
mtfa = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/messages_to_federal_assembly.pkl')
mtfa

,date,URL,title,speaker,text
0,2020-01-15,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
1,2015-12-03,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...
2,2013-12-12,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
3,2012-12-12,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
4,2016-12-01,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...
5,2018-03-01,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
6,2009-11-12,http://kremlin.ru/events/president/transcripts...,Послание Федеральному Собранию Российской Феде...,Д.Медведев:,Все материалы сайта доступны по лицензии: Crea...
7,2021-04-21,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...
8,2014-12-04,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
9,2019-02-20,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,"* * *, В.Путин:",Все материалы сайта доступны по лицензии: Crea...
